In [6]:
import unittest
import math
import os
import numpy as np
import time
import glob


import pptk
import cv2
from pano import GSV_pano
from PIL import Image
import PIL

# gsv = GSV_pano()
import matplotlib.pyplot as plt

In [7]:
seg_file = r'--69cR9y-yjGxq3c-uPBRw.png'

panoId = seg_file[:-4]

saved_path = r'D:\Code\StreetView\gsv_pano\test_results'

pano1 = GSV_pano(panoId=panoId, crs_local=6487, saved_path=saved_path)


In [10]:
seg_file = r'--69cR9y-yjGxq3c-uPBRw.png'
panoId = seg_file[:-4]
saved_path = r'D:\Code\StreetView\gsv_pano\test_results'

rows_offset = 2048
cols_offset = 0
zoom = 4

pil_img = Image.open(seg_file)

pano1 = GSV_pano(panoId=panoId, crs_local=6487, saved_path=saved_path)

# DOM = pano1.get_DOM(zoom=4, resolution=0.05, fill_clipped_seg=True)

target_ids = [12]
np_img = np.array(pil_img)

np_img_binary = np.zeros(np_img.shape)
for i in target_ids:
    np_img_binary = np.logical_or(np_img_binary, np_img == i)

np_img_binary = np_img_binary.astype(np.uint8)

# cv2_opened = np.where(cv2_opened == 0, 0, 255).astype(np.uint8)

# opened_color = cv2.merge((cv2_opened, cv2_opened, cv2_opened))

morph_kernel_open  = (10, 10)
morph_kernel_close = (20, 20)
g_close = cv2.getStructuringElement(cv2.MORPH_RECT, morph_kernel_close)
g_open  = cv2.getStructuringElement(cv2.MORPH_RECT, morph_kernel_open)

cv2_img_closed = cv2.morphologyEx(np_img_binary, cv2.MORPH_CLOSE, g_close) # fill small gaps
cv2_img_opened = cv2.morphologyEx(cv2_img_closed, cv2.MORPH_OPEN, g_open)

raw_contours, hierarchy = cv2.findContours(cv2_img_opened.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

cv_img_color = cv2.cvtColor(np_img, cv2.COLOR_GRAY2BGR)
# cv2.drawContours(cv_img_color, raw_contours, )
cv_img_con = cv2.drawContours(cv_img_color, raw_contours[18:19], -1, (0, 255, 0), 2)

# contours = [np.squeeze(cont) for cont in raw_contours[18:19]]
contours = [np.squeeze(cont) for cont in raw_contours[:]]


for idx, contour in enumerate(contours):
    cols = contour[:, 0] + cols_offset
    rows = contour[:, 1] + rows_offset
    contour_points = pano1.col_row_to_points(cols, rows, zoom=zoom)
    print("contour_points:", contour_points)

    for x, y in zip(contour_points[:, 0] + 20,  20-contour_points[:, 1]):
        cv2.circle(cv_img_con, (int(x*30), int(y*30)), 2, (0, 255, 0), 2)

win_name = "opencv"
cv2.namedWindow(win_name, cv2.WINDOW_NORMAL)
cv2.moveWindow(win_name, 100, 100)
cv2.imshow(win_name, cv_img_con)
cv2.resizeWindow(win_name, 1600, 200)
cv2.waitKey(0)


contour_points: [[ 1.46295221 -2.10348187 -2.42358001  3.52684069]
 [ 1.46065604 -2.10014447 -2.4234684   3.52382207]
 [ 1.4583608  -2.09680847 -2.42335201  3.52080345]
 [ 1.45606649 -2.09347388 -2.42323083  3.51778483]
 [ 1.45379638 -2.09017415 -2.42314366  3.51482248]
 [ 1.45155064 -2.0869095  -2.42309092  3.51191688]
 [ 1.44930567 -2.08364602 -2.42303337  3.50901103]
 [ 1.44706168 -2.08038401 -2.42297134  3.50610542]
 [ 1.44481857 -2.07712333 -2.42290465  3.50319982]
 [ 1.44257624 -2.07386383 -2.42283316  3.50029397]
 [ 1.44033489 -2.0706058  -2.42275719  3.49738836]
 [ 1.43809434 -2.06734897 -2.42267641  3.49448252]
 [ 1.43585477 -2.06409362 -2.42259116  3.49157691]
 [ 1.433616   -2.06083947 -2.42250112  3.48867106]
 [ 1.43137822 -2.05758681 -2.4224066   3.48576546]
 [ 1.42914124 -2.05433536 -2.42230729  3.48285961]
 [ 1.42690527 -2.05108541 -2.42220352  3.479954  ]
 [ 1.4246702  -2.04783682 -2.42209512  3.4770484 ]
 [ 1.42243593 -2.04458945 -2.42198193  3.47414255]
 [ 1.4208391  -

 [-13.44589589  10.50486189  -2.26069371  17.21205902]]
contour_points: [[-16.01011533 -11.3405307   -2.63846456  19.79628563]
 [-15.88575343 -11.27065173  -2.63468641  19.6551857 ]
 [-15.75374813 -11.19507572  -2.6293934   19.50446129]
 [-15.74640055 -11.20805382  -2.62969088  19.5060215 ]
 [-15.61443333 -11.13208026  -2.62413936  19.35509872]
 [-15.60708188 -11.14491711  -2.6244287   19.35659599]
 [-15.47515404 -11.068546    -2.61861819  19.20547485]
 [-15.35060575 -10.97935703  -2.61228518  19.05285835]
 [-15.22607372 -10.8901814   -2.60571874  18.90024185]
 [-15.10155818 -10.80101927  -2.59891889  18.74762535]
 [-14.97705935 -10.71187079  -2.59188567  18.59500885]
 [-14.85257591 -10.62273502  -2.58461884  18.44239044]
 [-14.72811115 -10.53361432  -2.57711896  18.28977394]
 [-14.60366375 -10.44450773  -2.56938578  18.13715744]
 [-14.47923393 -10.35541542  -2.56141936  17.98454094]
 [-14.3548219  -10.26633754  -2.5532197   17.83192444]
 [-14.22322691 -10.18863511  -2.54497725  17.680

-1